In [1]:
from scipy.linalg import eigh
from openfermion.transforms import get_sparse_operator
from projectq.ops import X
from projectq.backends import CommandPrinter
from openfermionprojectq import uccsd_trotter_engine, uccsd_singlet_evolution, uccsd_singlet_paramsize
from openfermion.utils import pauli_exp_to_qasm
from openfermion.ops import QubitOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from pyscf import ci
from pyscf import mp
from openfermionpyscf import run_pyscf
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4
import os

from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize

from openfermion.config import *
from openfermionprojectq import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize

from projectq.ops import X, All, Measure
from projectq.backends import CommandPrinter, CircuitDrawer

In [2]:
#menentukan molekul yang akan ditinjau
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.74]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [3]:
#informasi-informasi dari molekul
one_electron_integrals = h2_molecule.one_body_integrals
two_electron_integrals = h2_molecule.two_body_integrals
orbitals = h2_molecule.canonical_orbitals
qubit= h2_molecule.n_qubits
overlap=h2_molecule.overlap_integrals

#print(qubit)
#print("Energi Hartree=", h2_molecule.hf_energy)
#print("Energi FCI=", h2_molecule.fci_energy)
#print("Energi MP2=", h2_molecule.mp2_energy)
#print("koefisien h_pq=",one_electron_integrals)
#print("koefisien h_pqrs=",two_electron_integrals)
#print("koefisien orbitals =",orbitals)
#print("overlap=",overlap)

In [4]:
qubit_hamiltonian = jordan_wigner(h2_molecule.get_molecular_hamiltonian())
#print(qubit_hamiltonian)
qubit_hamiltonian.compress()
compiler_engine = uccsd_trotter_engine()

In [5]:
#membuat fungsi energi untuk menentukan tebakan amplitudo yang paling sesuai

def energy_objective(packed_amplitudes):
    

    wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)
    for i in range(h2_molecule.n_electrons):
        X | wavefunction[i]

    
    evolution_operator = uccsd_singlet_evolution(packed_amplitudes, 
                                                 h2_molecule.n_qubits, 
                                                 h2_molecule.n_electrons)
    evolution_operator | wavefunction
    compiler_engine.flush()

    
    energy = compiler_engine.backend.get_expectation_value(qubit_hamiltonian, wavefunction)
    All(Measure) | wavefunction
    compiler_engine.flush()
    return energy

In [6]:
n_amplitudes = uccsd_singlet_paramsize(molecule.n_qubits, molecule.n_electrons) #banyak parameter nilai coba
print(n_amplitudes)

2


In [7]:
import numpy as np
compiler_engine = uccsd_trotter_engine()
wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)


In [8]:
import numpy as np
initial_amplitudes = np.linspace(0,0.05677,2)
uccsd=uccsd_singlet_evolution(initial_amplitudes,h2_molecule.n_qubits, h2_molecule.n_electrons)
print(uccsd)
print(initial_amplitudes)

exp(-1j * (0.0141925 Y0 Y1 Y2 X3 +
-0.0141925 Y0 X1 Y2 Y3 +
-0.0141925 X0 X1 Y2 X3 +
-0.0141925 X0 Y1 Y2 Y3 +
0.0141925 Y0 X1 X2 X3 +
0.0141925 Y0 Y1 X2 Y3 +
0.0141925 X0 Y1 X2 X3 +
-0.0141925 X0 X1 X2 Y3))
[0.      0.05677]


In [9]:
import numpy as np
initial_amplitudes = np.linspace(0,0.05,2)  #jumlah anggota array nya disesuaikan
initial_energy = energy_objective(initial_amplitudes)

#optimasi untuk mencari parameter untuk mencari ground state
opt_result = minimize(energy_objective, initial_amplitudes,
                      method="COBYLA", options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print("\nOptimal UCCSD Singlet Energi: {}".format(opt_energy))
print("Optimal UCCSD Singlet Amplitud0: {}".format(opt_amplitudes))
print("Klasik CCSD Energy: {} Hartrees".format(h2_molecule.ccsd_energy))
print("Eksak FCI Energy: {} Hartrees".format(h2_molecule.fci_energy))
print("Energi Mula-Mula {} Hartrees".format(initial_energy))
print("Energi HF {} Hartrees".format(h2_molecule.hf_energy))


Optimal UCCSD Singlet Energi: -1.13728381008158
Optimal UCCSD Singlet Amplitud0: [-1.21402783e-05  5.64523804e-02]
Klasik CCSD Energy: -1.1372839986104393 Hartrees
Eksak FCI Energy: -1.1372838344885023 Hartrees
Energi Mula-Mula -1.1370190698544984 Hartrees
Energi HF -1.1167593073964248 Hartrees


In [28]:
opt_result = minimize(energy_objective, initial_amplitudes,
                      method="CG", options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print("\nOptimal UCCSD Singlet Energi: {}".format(opt_energy))
print("Optimal UCCSD Singlet Amplitud0: {}".format(opt_amplitudes))
print("Klasik CCSD Energy: {} Hartrees".format(h2_molecule.ccsd_energy))
print("Eksak FCI Energy: {} Hartrees".format(h2_molecule.fci_energy))
print("Energi Mula-Mula {} Hartrees".format(initial_energy))
print("Energi HF {} Hartrees".format(h2_molecule.hf_energy))

RuntimeError: AllocateQubit: ID already exists. Qubit IDs should be unique.
 raised in:
'  File "/home/muhammadaugi/yes/envs/openfermion/lib/python3.7/site-packages/projectq/backends/_sim/_simulator.py", line 382, in _handle'
'    self._simulator.allocate_qubit(ID)'

In [11]:
opt_result = minimize(energy_objective, initial_amplitudes,
                      method="Powell", options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print(opt_amplitudes)
print("\nOptimal UCCSD Singlet Energi: {}".format(opt_energy))
print("Optimal UCCSD Singlet Amplitud0: {}".format(opt_amplitudes))
print("Klasik CCSD Energy: {} Hartrees".format(h2_molecule.ccsd_energy))
print("Eksak FCI Energy: {} Hartrees".format(h2_molecule.fci_energy))
print("Energi Mula-Mula {} Hartrees".format(initial_energy))
print("Energi HF {} Hartrees".format(h2_molecule.hf_energy))

Optimization terminated successfully.
         Current function value: -1.137284
         Iterations: 2
         Function evaluations: 71
[2.67921476e-11 5.63914170e-02]

Optimal UCCSD Singlet Energi: -1.1372838344884957
Optimal UCCSD Singlet Amplitud0: [2.67921476e-11 5.63914170e-02]
Klasik CCSD Energy: -1.1372839986104395 Hartrees
Eksak FCI Energy: -1.137283834488502 Hartrees
Energi Mula-Mula -1.1370190698544984 Hartrees
Energi HF -1.116759307396425 Hartrees


In [10]:
opt_result = minimize(energy_objective, initial_amplitudes,
                      method="Nelder-Mead", options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print(opt_amplitudes)
print("\nOptimal UCCSD Singlet Energi: {}".format(opt_energy))
print("Optimal UCCSD Singlet Amplitud0: {}".format(opt_amplitudes))
print("Klasik CCSD Energy: {} Hartrees".format(h2_molecule.ccsd_energy))
print("Eksak FCI Energy: {} Hartrees".format(h2_molecule.fci_energy))
print("Energi Mula-Mula {} Hartrees".format(initial_energy))
print("Energi HF {} Hartrees".format(h2_molecule.hf_energy))

Optimization terminated successfully.
         Current function value: -1.137284
         Iterations: 10
         Function evaluations: 20
[1.56250000e-05 5.64151875e-02]

Optimal UCCSD Singlet Energi: -1.1372838303003543
Optimal UCCSD Singlet Amplitud0: [1.56250000e-05 5.64151875e-02]
Klasik CCSD Energy: -1.1372839986104415 Hartrees
Eksak FCI Energy: -1.137283834488502 Hartrees
Energi Mula-Mula -1.1372829054972706 Hartrees
Energi HF -1.1167593073964255 Hartrees


In [12]:
#circuit dengan cobyla
compiler_engine = uccsd_trotter_engine(CommandPrinter())
wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)
amplitudo = [-0.00017861 , 0.05625669]
evolution_operator = uccsd_singlet_evolution(amplitudo, h2_molecule.n_qubits, h2_molecule.n_electrons)
print('Sample Output:')
for i in range(h2_molecule.n_electrons):
    X | wavefunction[i]
evolution_operator | wavefunction
compiler_engine.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
Sample Output:
X | Qureg[0]
X | Qureg[1]
Rx(1.570796326795) | Qureg[0]
H | Qureg[2]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
Rz(12.566192004359) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[2]
Rx(10.995574287564) | Qureg[0]
H | Qureg[0]
Rx(1.570796326795) | Qureg[2]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
Rz(0.00017861) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
Rx(10.995574287564) | Qureg[2]
H | Qureg[0]
Rx(1.570796326795) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
H | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(0.028128345) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[3]
Rx(10.995574287564) | Qureg[2]
Rx(10.995574287564) | Qureg[1]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) |

In [13]:
#circuit CG
compiler_engine = uccsd_trotter_engine(CommandPrinter())
wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)
amplitudo = [-1.49437501e-08  ,5.63913897e-02]
evolution_operator = uccsd_singlet_evolution(amplitudo, h2_molecule.n_qubits, h2_molecule.n_electrons)
print(evolution_operator)
print('Sample Output:')
for i in range(h2_molecule.n_electrons):
    X | wavefunction[i]
evolution_operator | wavefunction
compiler_engine.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
Deallocate | Qureg[3]
Deallocate | Qureg[2]
Deallocate | Qureg[1]
Deallocate | Qureg[0]
exp(-1j * (0.014097847425 Y0 Y1 Y2 X3 +
-0.014097847425 Y0 X1 Y2 Y3 +
-0.014097847425 X0 X1 Y2 X3 +
-0.014097847425 X0 Y1 Y2 Y3 +
0.014097847425 Y0 X1 X2 X3 +
0.014097847425 Y0 Y1 X2 Y3 +
0.014097847425 X0 Y1 X2 X3 +
-0.014097847425 X0 X1 X2 Y3))
Sample Output:
X | Qureg[0]
X | Qureg[1]
Rx(1.570796326795) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
H | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(0.02819569485) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[3]
Rx(10.995574287564) | Qureg[2]
Rx(10.995574287564) | Qureg[1]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) | Qureg[0]
H | Qureg[1]
Rx(1.570796326795) | Qureg[2]
Rx(1.570796326795) | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( 

In [14]:
#circuit dengan Powell
compiler_engine = uccsd_trotter_engine(CommandPrinter())
wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)
amplitudo = [0.     ,    0.05639141]
evolution_operator = uccsd_singlet_evolution(amplitudo, h2_molecule.n_qubits, h2_molecule.n_electrons)
print(evolution_operator)
print('Sample Output:')
for i in range(h2_molecule.n_electrons):
    X | wavefunction[i]
evolution_operator | wavefunction
compiler_engine.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
Deallocate | Qureg[3]
Deallocate | Qureg[2]
Deallocate | Qureg[1]
Deallocate | Qureg[0]
exp(-1j * (0.0140978525 Y0 Y1 Y2 X3 +
-0.0140978525 Y0 X1 Y2 Y3 +
-0.0140978525 X0 X1 Y2 X3 +
-0.0140978525 X0 Y1 Y2 Y3 +
0.0140978525 Y0 X1 X2 X3 +
0.0140978525 Y0 Y1 X2 Y3 +
0.0140978525 X0 Y1 X2 X3 +
-0.0140978525 X0 X1 X2 Y3))
Sample Output:
X | Qureg[0]
X | Qureg[1]
Rx(1.570796326795) | Qureg[0]
Rx(1.570796326795) | Qureg[1]
Rx(1.570796326795) | Qureg[2]
H | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[2], Qureg[3] )
Rz(0.028195705) | Qureg[3]
CX | ( Qureg[2], Qureg[3] )
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[3]
Rx(10.995574287564) | Qureg[2]
Rx(10.995574287564) | Qureg[1]
Rx(10.995574287564) | Qureg[0]
Rx(1.570796326795) | Qureg[0]
H | Qureg[1]
Rx(1.570796326795) | Qureg[2]
Rx(1.570796326795) | Qureg[3]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2]

In [15]:
#circuit dengan Nelder Mead
compiler_engine = uccsd_trotter_engine(CommandPrinter())
wavefunction = compiler_engine.allocate_qureg(h2_molecule.n_qubits)
amplitudo = [1.56250000e-05 ,5.64151875e-02]
evolution_operator = uccsd_singlet_evolution(amplitudo, h2_molecule.n_qubits, h2_molecule.n_electrons)
print(evolution_operator)
print('Sample Output:')
for i in range(h2_molecule.n_electrons):
    X | wavefunction[i]
evolution_operator | wavefunction
compiler_engine.flush()

Allocate | Qureg[0]
Allocate | Qureg[1]
Allocate | Qureg[2]
Allocate | Qureg[3]
Deallocate | Qureg[3]
Deallocate | Qureg[2]
Deallocate | Qureg[1]
Deallocate | Qureg[0]
exp(-1j * (7.8125e-06 Y0 Z1 X2 +
-7.8125e-06 X0 Z1 Y2 +
0.014103796875 Y0 Y1 Y2 X3 +
-0.014103796875 Y0 X1 Y2 Y3 +
-0.014103796875 X0 X1 Y2 X3 +
-0.014103796875 X0 Y1 Y2 Y3 +
0.014103796875 Y0 X1 X2 X3 +
0.014103796875 Y0 Y1 X2 Y3 +
0.014103796875 X0 Y1 X2 X3 +
-0.014103796875 X0 X1 X2 Y3 +
7.8125e-06 Y1 Z2 X3 +
-7.8125e-06 X1 Z2 Y3))
Sample Output:
X | Qureg[0]
X | Qureg[1]
Rx(1.570796326795) | Qureg[0]
H | Qureg[2]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
Rz(1.5625e-05) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
H | Qureg[2]
Rx(10.995574287564) | Qureg[0]
H | Qureg[0]
Rx(1.570796326795) | Qureg[2]
CX | ( Qureg[0], Qureg[1] )
CX | ( Qureg[1], Qureg[2] )
Rz(12.566354989359) | Qureg[2]
CX | ( Qureg[1], Qureg[2] )
CX | ( Qureg[0], Qureg[1] )
Rx(10.995574287564) | Qureg[2]
H | Qureg[0]

In [25]:
fermion_hamiltonian = get_fermion_operator(h2_molecule.get_molecular_hamiltonian())
qubit_hamiltonian1 = bravyi_kitaev(fermion_hamiltonian)
print(qubit_hamiltonian1)




(-0.09706626816762814+0j) [] +
(0.04530261550379928+0j) [X0 Z1 X2] +
(0.04530261550379928+0j) [X0 Z1 X2 Z3] +
(0.04530261550379928+0j) [Y0 Z1 Y2] +
(0.04530261550379928+0j) [Y0 Z1 Y2 Z3] +
(0.17141282644776887+0j) [Z0] +
(0.17141282644776892+0j) [Z0 Z1] +
(0.16592785033770346+0j) [Z0 Z1 Z2] +
(0.16592785033770346+0j) [Z0 Z1 Z2 Z3] +
(0.12062523483390417+0j) [Z0 Z2] +
(0.12062523483390417+0j) [Z0 Z2 Z3] +
(0.16868898170361207+0j) [Z1] +
(-0.22343153690813583+0j) [Z1 Z2 Z3] +
(0.17441287612261608+0j) [Z1 Z3] +
(-0.2234315369081358+0j) [Z2]
